In [ ]:
# rename views
# rename channels
# rename data on S3
# change paths
# remove unused files from S3

In [1]:
# imports
import re
import pandas as pd
import numpy as np
import json
from typing import Any
import subprocess
from tqdm import tqdm

/tmp/ipykernel_64719/4237793295.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
COLORS = {
    "nhs": "white",
    "bod": "white",
    "tub": "magenta",
    "cetn": "yellow",
    "dna": "blue",
    "npc": "green",
}
ALT_COLORS = ["orange", "red"]

def get_colors_for_channels(channels: list[str]) -> list[str]:
    """
    Get colors for channels.
    """
    colors:list[str] = []
    alt_counter: int = 0
    for channel in channels:
        # use black for na
        if channel == "na":
            colors.append("black")
        # the only case for this is bod-dna-nhs-tub
        # therefore I can just use alternative colors since ornage is the first
        # one there
        elif channel == "bod" and "nhs" in channels:
            colors.append(ALT_COLORS[
                alt_counter
            ])
            alt_counter += 1
        elif channel in COLORS.keys():
            colors.append(COLORS[channel])
        else:
            colors.append(ALT_COLORS[
                alt_counter
            ])
            alt_counter += 1
        if alt_counter > len(ALT_COLORS):
            raise ValueError(f"Ran out of colors for {channels}")
    return colors

In [36]:
# load table & mobie dataset
table = pd.read_csv('final_name_map.csv', sep='\t')  # type: ignore
with open('/home/hellgoth/Documents/work/projects/culture-collections_project/culture-collections/data/single_volumes/dataset.json', 'r') as f:
    dataset = json.load(f)

In [38]:
# iterate over views to find all relevant files and create mapping for file names
unusded_files: list[str] = []
file_name_map: dict[str, str] = dict()
for view_name in dataset["views"]:
    if view_name == "default":
        continue
    assert view_name in table["Filename"].tolist(), f"{view_name} not in table"
    new_view_name: str = table.loc[  # type: ignore
        table["Filename"] == view_name, "new_filename_iteration4_manualRename"
    ].values[  # type: ignore
        0
    ]
    # iterate over all sourceDisplays
    for i, source_display in enumerate(dataset["views"][view_name]["sourceDisplays"]):
        in_if = False
        # get old file path
        source_name = source_display["imageDisplay"]["sources"][0]
        assert source_name in dataset["sources"], f"{source_name} not in dataset"
        source = dataset["sources"][source_name]
        old_file_name = (
            source["image"]["imageData"]["ome.zarr"]["relativePath"]
            .split("/")[2]
            .replace(".ome.zarr", "")
        )

        # add unused files to the remove list
        if not isinstance(new_view_name, str):
            assert np.isnan(new_view_name)  # type: ignore
            unusded_files.append(old_file_name)
            continue
        if new_view_name == "delete":
            unusded_files.append(old_file_name)
            continue
        if "DROPFILE" in new_view_name:
            unusded_files.append(old_file_name)
            continue
        if (
            table.loc[table["Filename"] == view_name, "remove"].values[0] == "x"  # type: ignore
        ):
            unusded_files.append(old_file_name)
            continue
        assert isinstance(new_view_name, str), f"{new_view_name} is not a string"

        # get new file path
        new_file_name = re.sub(r"_s\d+", "", new_view_name)
        if not source_name.split("_")[-1].startswith(
            "ch"
        ):  # Hiral's case, channels in separate files
            in_if = True
            view_name_split = view_name.split("_")
            new_view_name_split = new_view_name.split("_")
            new_channels = new_view_name_split[2].split("-")
            new_file_name = "_".join(
                new_view_name_split[:2]
                + new_channels[i : i + 1]
                + new_view_name_split[3:]
            )
        # if new file name already exists
        # and the current new file name is shorter than the already existing one (this is needed because
        # sometimes the channels changed for only one/a few volume(s) in a series)
        if old_file_name in file_name_map.keys() and len(new_file_name) <= len(file_name_map[old_file_name]):
            continue
        else:
            file_name_map[old_file_name] = new_file_name

for k, v in file_name_map.items():
    print(f"{k}: {v}")

20230423_RCC_1216_pfa_OD_nhs-tub-Hoechst: rcc1216_pfa_dna-tub-nhs_20230423_od
20230423_RCC_1534_pfa_OD_nhs-tub-Hoechst: rcc1534_pfa_nhs-tub-dna_20230423_od
20230423_RCC_1536_pfa_OD_nhs-tub-Hoechst: rcc1536_pfa_nhs-tub-dna_20230423_od
20230423_RCC_15_pfa_OD_nhs-tub-Hoechst: rcc15_pfa_nhs-tub-dna_20230423_od
20230423_RCC_1805_pfa_OD_nhs-tub-Hoechst: rcc1805_pfa_nhs-tub-dna_20230423_od
20230423_RCC_417_pfa_OD_nhs-tub-Hoechst: rcc417_pfa_nhs-tub-dna_20230423_od
20230423_RCC_4660_pfa_OD_nhs-tub-Hoechst: rcc4660_pfa_nhs-tub-dna_20230423_od
20230423_RCC_6124_pfa_OD_nhs-tub-Hoechst: rcc6124_pfa_dna-tub-nhs_20230423_od
20230423_RCC_6369_pfa_OD_nhs-tub-Hoechst: rcc6369_pfa_nhs-tub-dna_20230423_od
20230423_RCC_6716_pfa_OD_nhs-tub-Hoechst: rcc6716_pfa_nhs-tub-dna_20230423_od
20230423_RCC_AC269_pfa_OD_nhs-tub-Hoechst: rcc10349_pfa_nhs-tub-dna_20230423_od
20230423_RCC_ARC373_pfa_OD_nhs-tub-Hoechst: rccarc373_pfa_nhs-tub-dna_20230423_od
20230423_RCC_ARC47_pfa_OD_nhs-tub-Hoechst: rcc10360_pfa_nhs-tub-

In [40]:
uniques, counts = np.unique(list(file_name_map.values()), return_counts=True)
counts

array([1, 1, 1, ..., 1, 1, 1])

In [41]:
uniques[counts > 1]

array([], dtype='<U51')

In [42]:
actually_unused_files: list[str] = []
for name in unusded_files:
    if name not in file_name_map.keys():
        actually_unused_files.append(name)
        print(name)

20230427_rcck1260_pfa_hoechst-tub-centr-3
20230712_3598+N1044_PFA-NHS_e7_DNA_001
20230712_3598+N1044_PFA-NHS_e7_DNA_001
20230712_3598+N1044_PFA-NHS_e7_DNA_001
20230712_vf21-45+1200_pfa-nhs_e7_dna_001
20230712_vf21-45+1200_pfa-nhs_e7_dna_001
20230712_vf21-45+1200_pfa-nhs_e7_dna_001
20230824_tokyo_chem_NHS-tub-DNA-Airyscan_Processing-05
20231025_rccc2471_pfa_centr-tub-hoechst
20231025_rccc2471_pfa_centr-tub-hoechst
20231025_rccc2471_pfa_centr-tub-hoechst
20231025_rccc2471_pfa_centr-tub-hoechst
20231025_rccc2471_pfa_centr-tub-hoechst
20231025_rccc2471_pfa_centr-tub-hoechst
20231025_rccc2471_pfa_centr-tub-hoechst
20231025_rccc2471_pfa_centr-tub-hoechst
20231025_rccc2471_pfa_centr-tub-hoechst
20240212_RCC6327_DNA-tub-cent-nhs_001
20240212_RCC6327_DNA-tub-cent-nhs_001
20240212_RCC6327_DNA-tub-cent-nhs_001
20240212_RCC6327_DNA-tub-cent-nhs_001
20240212_RCC6327_DNA-tub-cent-nhs_003
20240212_RCC6327_DNA-tub-cent-nhs_003
20240212_RCC6327_DNA-tub-cent-nhs_003
20240212_RCC6327_DNA-tub-cent-nhs_003

In [43]:
len(unusded_files), len(actually_unused_files)

(395, 104)

In [44]:
# save both to file
with open("files_to_remove.csv", "w") as f:
    for name in actually_unused_files:
        f.write(f"{name}\n")
with open("file_name_map.csv", "w") as f:
    for k, v in file_name_map.items():
        f.write(f"{k},{v}\n")

In [46]:
# # remove unused files
# for name in tqdm(actually_unused_files):
#     cmd = ["mc", "rm", "-r", "--force", f"culcol_s3_rw/culture-collections/data/single_volumes/images/ome-zarr/{name}.ome.zarr"]
#     subprocess.run(cmd)

In [47]:
# # rename files
# for old_file_name, new_file_name in tqdm(file_name_map.items(), total=len(file_name_map)):
#     cmd = [
#         "mc",
#         "mv",
#         f"culcol_s3_rw/culture-collections/data/single_volumes/images/ome-zarr/{old_file_name}.ome.zarr",
#         f"culcol_s3_rw/culture-collections/data/single_volumes/images/ome-zarr/{new_file_name}.ome.zarr",
#     ]
#     subprocess.run(cmd)

In [48]:
# create new table
new_views = {}
new_sources = {}
# iterate over all views excluding default
for view_name in dataset["views"]:
    if view_name == "default":
        continue
    assert view_name in table["Filename"].tolist(), f"{view_name} not in table"
    new_view_name: str = table.loc[  # type: ignore
        table["Filename"] == view_name, "new_filename_iteration4_manualRename"
    ].values[  # type: ignore
        0
    ]
    # ignore files that are supposed to be deleted
    if not isinstance(new_view_name, str):
        assert np.isnan(new_view_name)  # type: ignore
        continue
    if new_view_name == "delete":
        continue
    if "DROPFILE" in new_view_name:
        continue
    if (
        table.loc[table["Filename"] == view_name, "remove"].values[0] == "x"  # type: ignore
    ):
        continue
    assert isinstance(new_view_name, str), f"{new_view_name} is not a string"
    view_name_split = view_name.split("_")
    channels = view_name_split[-2].split("-")
    new_view_name_split = new_view_name.split("_")
    new_channels = new_view_name_split[2].split("-")
    colors = get_colors_for_channels(new_channels)
    # iterate over all attached source_displays = channels
    assert (
        len(new_channels)
        == len(colors)
        == len(dataset["views"][view_name]["sourceDisplays"])
    ), f"{len(new_channels)} != {len(colors)} != {len(dataset['views'][view_name]['sourceDisplays'])} for {view_name} {new_view_name}"
    new_source_displays: list[dict[str, Any]] = []
    for i, source_display in enumerate(dataset["views"][view_name]["sourceDisplays"]):
        # replace paths in source
        source_name = source_display["imageDisplay"]["sources"][0]
        assert source_name in dataset["sources"], f"{source_name} not in dataset"
        source = dataset["sources"][source_name]
        # old_file_name = (
        #         source["image"]["imageData"]["ome.zarr"]["relativePath"]
        #         .split("/")[2]
        #         .replace(".ome.zarr", "")
        #     )
        # assert old_file_name in file_name_map.keys(), f"{old_file_name} not in file_name_map"
        # new_file_name = file_name_map[old_file_name]
        # source["image"]["imageData"]["ome.zarr"]["relativePath"] = source["image"][
        #     "imageData"
        # ]["ome.zarr"]["relativePath"].replace(old_file_name, new_file_name)
        # source["image"]["imageData"]["ome.zarr.s3"]["s3Address"] = source["image"][
        #     "imageData"
        # ]["ome.zarr.s3"]["s3Address"].replace(old_file_name, new_file_name)
        # get new source name
        if source_name.split("_")[-1].startswith("ch"):  # normal case
            new_source_name = new_view_name + f"_ch{i}"
        else:  # Hiral's case, channels in separate files
            new_source_name = "_".join(
                new_view_name_split[:2] + new_channels[i : i + 1] + new_view_name_split[3:]
            )
        # add new source to new sources
        new_sources[new_source_name] = source
        # correct channel names, use correct colors and change source_display name
        new_source_display: dict[str, Any] = source_display.copy()
        new_source_display["imageDisplay"]["color"] = colors[i]
        new_source_display["imageDisplay"]["name"] = new_channels[i]
        new_source_display["imageDisplay"]["sources"][0] = new_source_name
        new_source_displays.append(new_source_display)
    # replace source_displays in view
    new_view: dict[str, Any] = dataset["views"][view_name].copy()
    new_view["sourceDisplays"] = new_source_displays
    new_views[new_view_name] = new_view
# create new dataset
new_dataset: dict[str, Any] = dataset.copy()
new_dataset["is2D"] = False
new_dataset["sources"] = new_sources
new_dataset["views"] = new_views

In [50]:
# add default view
new_dataset["views"]["default"] = dataset["views"]["default"]

In [51]:
import os

p = "/home/hellgoth/Documents/work/projects/culture-collections_project/culture-collections/data/new_names/dataset.json"
os.makedirs(os.path.dirname(p), exist_ok=True)
with open(p, "w") as f:
    json.dump(new_dataset, f, indent=2)

In [ ]:
# check which vols contain na channel
for view_name in dataset["views"]:
    if view_name == "default":
        continue
    assert view_name in table["Filename"].tolist(), f"{view_name} not in table"
    new_view_name: str = table.loc[  # type: ignore
        table["Filename"] == view_name, "new_filename_iteration4_manualRename"
    ].values[  # type: ignore
        0
    ]
    # ignore files that are supposed to be deleted
    if not isinstance(new_view_name, str):
        assert np.isnan(new_view_name)  # type: ignore
        continue
    if new_view_name == "delete":
        continue
    if "DROPFILE" in new_view_name:
        continue
    if (
        table.loc[table["Filename"] == view_name, "remove"].values[0] == "x"  # type: ignore
    ):
        continue
    assert isinstance(new_view_name, str), f"{new_view_name} is not a string"
    new_view_name_split = new_view_name.split("_")
    new_channels = new_view_name_split[2].split("-")
    if "na" in new_channels:
        print(view_name, new_view_name)

20230718_rcc_1521_pfa_csd_hoechst_tub_nhs_na_01 rcc1521_pfa_dna-tub-nhs-na_20230718_csd_01
20230718_rcc_1521_pfa_csd_hoechst_tub_nhs_na_02 rcc1521_pfa_dna-tub-nhs-na_20230718_csd_02
20230718_rcc_3536_pfa_csd_hoechst_tub_nha_na_01 rcc3536_pfa_dna-tub-nhs-na_20230718_csd_01
20230718_rcc_3536_pfa_csd_hoechst_tub_nha_na_02 rcc3536_pfa_dna-tub-nhs-na_20230718_csd_02
20230718_rcc_3536_pfa_csd_hoechst_tub_nha_na_03 rcc3536_pfa_dna-tub-nhs-na_20230718_csd_03
20230718_rcc_3536_pfa_csd_hoechst_tub_nha_na_04 rcc3536_pfa_dna-tub-nhs-na_20230718_csd_04
20230718_rcc_3536_pfa_csd_hoechst_tub_nha_na_05 rcc3536_pfa_dna-tub-nhs-na_20230718_csd_05
20230718_rcc_3536_pfa_csd_hoechst_tub_nha_na_06 rcc3536_pfa_dna-tub-nhs-na_20230718_csd_06
20230718_rcc_P21-B2_pfa_csd_hoechst_tub_nhs_na_01 rcc6387_pfa_dna-tub-nhs-na_20230718_csd_01
20230718_rcc_P21-B2_pfa_csd_hoechst_tub_nhs_na_02 rcc6387_pfa_dna-tub-nhs-na_20230718_csd_02
20230718_rcc_P21-B2_pfa_csd_hoechst_tub_nhs_na_03 rcc6387_pfa_dna-tub-nhs-na_20230718_

In [ ]:
# check if all channels are in order
for view_name in dataset["views"]:
    if view_name == "default":
        continue
    assert view_name in table["Filename"].tolist(), f"{view_name} not in table"
    new_view_name: str = table.loc[  # type: ignore
        table["Filename"] == view_name, "new_filename_iteration4_manualRename"
    ].values[  # type: ignore
        0
    ]
    if not isinstance(new_view_name, str):
        assert np.isnan(new_view_name)  # type: ignore
        continue
    if new_view_name == "delete":
        continue
    if "DROPFILE" in new_view_name:
        continue
    assert isinstance(new_view_name, str), f"{new_view_name} is not a string"
    new_view_name_split = new_view_name.split("_")
    new_channels = new_view_name_split[2].split("-")
    if (
        dataset["views"][view_name]["sourceDisplays"][0]["imageDisplay"]["sources"][0]
        .split("_")[-1]
        .startswith("ch")
    ):
        for i, source_display in enumerate(
            dataset["views"][view_name]["sourceDisplays"]
        ):
            source_name = source_display["imageDisplay"]["sources"][0]
            assert source_name in dataset["sources"], f"{source_name} not in dataset"
            assert source_name.split("_")[-1].startswith(
                "ch"
            ), f"{source_name} is not a channel"
            assert (
                int(source_name.split("_")[-1].replace("ch", "")) == i
            ), f"{source_name} is not in order"
    # should be in order in any case, also hard to check because hard to extract
    # channels from old name and order is changed in new names
    # else:  # Hiral's case, channels in separate files
    #     for i, source_display in enumerate(dataset["views"][view_name]["sourceDisplays"]):
    #         source_name = source_display["imageDisplay"]["sources"][0]
    #         assert channels[i] in source_name, f"{source_name} is not in order"

In [ ]:
# check how to filter for Hiral's case
for view_name in dataset["views"]:
    if view_name == "default":
        continue
    for source_display in dataset["views"][view_name]["sourceDisplays"]:
        source_name = source_display["imageDisplay"]["sources"][0]
        if not source_name.split("_")[-1].startswith("ch"):
            print(source_name)

20240212_bmcc_33_pfa_hs_cent_01
20240212_bmcc_33_pfa_hs_dna_01
20240212_bmcc_33_pfa_hs_nhs_01
20240212_bmcc_33_pfa_hs_tub_01
20240212_bmcc_33_pfa_hs_cent_02
20240212_bmcc_33_pfa_hs_dna_02
20240212_bmcc_33_pfa_hs_nhs_02
20240212_bmcc_33_pfa_hs_tub_02
20240212_bmcc_33_pfa_hs_cent_03
20240212_bmcc_33_pfa_hs_dna_03
20240212_bmcc_33_pfa_hs_nhs_03
20240212_bmcc_33_pfa_hs_tub_03
20240212_bmcc_33_pfa_hs_cent_04
20240212_bmcc_33_pfa_hs_dna_04
20240212_bmcc_33_pfa_hs_nhs_04
20240212_bmcc_33_pfa_hs_tub_04
20240212_bmcc_868_hs_pfa_cent_01
20240212_bmcc_868_hs_pfa_dna_01
20240212_bmcc_868_hs_pfa_nhs_01
20240212_bmcc_868_hs_pfa_tub_01
20240212_bmcc_868_hs_pfa_cent_02
20240212_bmcc_868_hs_pfa_dna_02
20240212_bmcc_868_hs_pfa_nhs_02
20240212_bmcc_868_hs_pfa_tub_02
20240212_bmcc_868_hs_pfa_cent_03
20240212_bmcc_868_hs_pfa_dna_03
20240212_bmcc_868_hs_pfa_nhs_03
20240212_bmcc_868_hs_pfa_tub_03
20240212_bmcc_868_hs_pfa_cent_04
20240212_bmcc_868_hs_pfa_dna_04
20240212_bmcc_868_hs_pfa_nhs_04
20240212_bmcc_86